In [1]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, MetaData, Table, Column, Float, Integer, String, extract, distinct, ForeignKey
from sqlalchemy.engine import reflection
from config import engine_url
import pandas as pd
import datetime as dt
from sqlalchemy.orm import relationship

<h2>Data Modeling</h2>

In [70]:
# Check EmployeeSQL/ER.png

<h2>Data Engineering</h2>

In [71]:
# Check EmployeeSQL/schema.sql

In [72]:
# Import each CSV file into the corresponding SQL table

In [2]:
engine = create_engine(engine_url)
connection = engine.connect()

In [18]:
metadata = MetaData()
metadata.reflect(engine, only=['employees', 'titles', 'departments', 'salaries'])

Table('dept_emp', metadata,
    Column('emp_no', Integer, primary_key=True),
    Column('dept_no', String, primary_key=True)
)

Table('dept_manager', metadata,
    Column('dept_no', String, primary_key=True),
    Column('emp_no', Integer, primary_key=True)
)

Base = automap_base(metadata=metadata)
Base.prepare()

In [3]:
#
metadata = MetaData()
metadata.reflect(engine, only=['employees', 'titles', 'departments', 'salaries'])

Table('dept_emp', metadata,
    Column('emp_no', Integer, primary_key=True),
    Column('dept_no', String, primary_key=True)
)

Table('dept_manager', metadata,
    Column('dept_no', String, ForeignKey("departments.dept_no"), primary_key=True),
    Column('emp_no', Integer, ForeignKey("employees.emp_no"), primary_key=True)
)

Base = automap_base(metadata=metadata)
Base.prepare()

In [4]:
# Check imported tables
for name in Base.classes.keys():
    print(name)

employees
titles
departments
salaries
dept_emp


In [11]:
Departments = Base.classes.departments
#Dept_emp = Base.classes.dept_emp
#Dept_manager = Base.classes.dept_manager
Employees = Base.classes.employees
Salaries = Base.classes.salaries
Titles = Base.classes.titles

In [21]:
# Check imported columns for an specific table
table_name = "salaries"
inspector = reflection.Inspector.from_engine(engine)
columns = inspector.get_columns(table_name=table_name)

for c in columns:
    print(c['name'], c["type"])

emp_no INTEGER
salary INTEGER


In [22]:
# Open session
session = Session(engine)

In [14]:
# Read all data 
data_departments = pd.read_csv("./data/departments.csv")
data_titles = pd.read_csv("./data/titles.csv")
data_employees = pd.read_csv("./data/employees.csv")
data_dept_emp = pd.read_csv("./data/dept_emp.csv")
data_dept_manager = pd.read_csv("./data/dept_manager.csv")
data_salaries = pd.read_csv("./data/salaries.csv")


In [63]:
# Inspect one of them
data_salaries.head(2)

,emp_no,salary
0,10001,60117
1,10002,65828


In [64]:
# Ckeck imported columns
for col in Salaries.__table__.columns.keys():
    print(col)

emp_no
salary


In [29]:
# Import each department to db
for index, row in data_departments.iterrows():
    #print(row["dept_no"], row["dept_name"])
    session.add(Departments(dept_no=row["dept_no"], dept_name=row["dept_name"]))

session.commit()

In [27]:
# Import each title to db
for index, row in data_titles.iterrows():
    session.add(Titles(title_id=row["title_id"], title=row["title"]))

session.commit()

In [39]:
# Import each employee to db
for index, row in data_employees.iterrows():
    #print(row["emp_no"], row["emp_title_id"])
    session.add(Employees(
        emp_no=row["emp_no"], 
        emp_title_id=row["emp_title_id"], 
        birth_date=dt.datetime.strptime(row["birth_date"], '%m/%d/%Y'), 
        first_name=row["first_name"], 
        last_name=row["last_name"], 
        sex=row["sex"], 
        hire_date=dt.datetime.strptime(row["hire_date"], '%m/%d/%Y')
    ))

session.commit()

In [43]:
# Import dept_emp to db
for index, row in data_dept_emp.iterrows():
    session.add(Dept_emp(emp_no=row["emp_no"], dept_no=row["dept_no"]))

session.commit()

In [46]:
# Import dept_manager to db
for index, row in data_dept_manager.iterrows():
    session.add(Dept_manager(dept_no=row["dept_no"], emp_no=row["emp_no"]))

session.commit()

In [57]:
data_salaries.dtypes

emp_no    int64
salary    int64
dtype: object

In [68]:
# Import salaries to db
for index, row in data_salaries.iterrows():
    session.add(Salaries(emp_no=int(row["emp_no"]), salary=int(row["salary"])))

session.commit()

<h2>Data Analysis</h2>

In [20]:
conn = engine.connect()

In [57]:
# List the following details of each employee: employee number, last name, first name, sex, and salary.
results = session.query(
    Employees.emp_no, 
    Employees.last_name, 
    Employees.first_name, 
    Employees.sex, 
    Employees.sex, 
    Salaries.salary
).filter(
    Employees.emp_no == Salaries.emp_no
).limit(10).all()

for result in results:
    print(result)

(10001, 'Facello', 'Georgi', 'M', 'M', 60117)
(10002, 'Simmel', 'Bezalel', 'F', 'F', 65828)
(10003, 'Bamford', 'Parto', 'M', 'M', 40006)
(10004, 'Koblick', 'Chirstian', 'M', 'M', 40054)
(10005, 'Maliniak', 'Kyoichi', 'M', 'M', 78228)
(10006, 'Preusig', 'Anneke', 'F', 'F', 40000)
(10007, 'Zielinski', 'Tzvetan', 'F', 'F', 56724)
(10008, 'Kalloufi', 'Saniya', 'M', 'M', 46671)
(10009, 'Peac', 'Sumant', 'F', 'F', 60929)
(10010, 'Piveteau', 'Duangkaew', 'F', 'F', 72488)


In [58]:
# List first name, last name, and hire date for employees who were hired in 1986.
results = session.query(
    Employees.first_name, 
    Employees.last_name, 
    Employees.hire_date, 
).filter(
    extract('year', Employees.hire_date) == 1986
).limit(10).all()

for result in results:
    print(result)

('Eran', 'Cusworth', datetime.date(1986, 11, 14))
('Bojan', 'Zallocco', datetime.date(1986, 10, 14))
('Nevio', 'Demizu', datetime.date(1986, 5, 18))
('Ziva', 'Vecchi', datetime.date(1986, 7, 3))
('Mohit', 'Speek', datetime.date(1986, 1, 14))
('Qunsheng', 'Speer', datetime.date(1986, 2, 13))
('Dines', 'Encarnacion', datetime.date(1986, 8, 2))
('Harngdar', 'Swick', datetime.date(1986, 5, 28))
('Freyja', 'Uhrig', datetime.date(1986, 12, 20))
('Zhenhua', 'Milicic', datetime.date(1986, 8, 4))


In [36]:
# List the department of each employee with the following information: 
#  employee number, last name, first name, and department name.
results = session.query(
    Employees.emp_no, 
    Employees.last_name, 
    Employees.first_name,
    Departments.dept_name
).filter(
    Dept_emp.dept_no == Departments.dept_no and 
    Dept_emp.emp_no == Employees.emp_no
).limit(10).all()

for result in results:
    print(result)

(473302, 'Zallocco', 'Hideyuki', 'Development')
(475053, 'Delgrande', 'Byong', 'Development')
(57444, 'Babb', 'Berry', 'Development')
(421786, 'Verhoeff', 'Xiong', 'Development')
(282238, 'Baumann', 'Abdelkader', 'Development')
(263976, 'Cusworth', 'Eran', 'Development')
(273487, 'Parfitt', 'Christoph', 'Development')
(461591, 'Samarati', 'Xudong', 'Development')
(477657, 'Magliocco', 'Lihong', 'Development')
(219881, 'Speek', 'Kwangyoen', 'Development')


In [15]:
session.close()

In [56]:
session.rollback()

In [13]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['employees', 'titles', 'departments', 'salaries']

In [14]:
# create a session
session = Session(engine)

In [16]:
Departments = Base.classes.departments
#Dept_emp = Base.classes.dept_emp
#Dept_manager = Base.classes.dept_manager
Employees = Base.classes.employees
Salaries = Base.classes.salaries
Titles = Base.classes.titles

In [27]:
# 4. List the manager of each department with the following information: 
#  department number, department name, the manager's employee number, last name, first name.
query_4 = """
SELECT d.dept_no, dp.emp_no, e.first_name, e.last_name 
FROM departments d
    JOIN dept_manager dp ON d.dept_no = dp.dept_no 
    JOIN employees e ON e.emp_no = dp.emp_no
GROUP BY d.dept_no, dp.emp_no, e.emp_no;
"""

data_4 = pd.read_sql(query_4, conn)
data_4.head(3)

,dept_no,emp_no,first_name,last_name
0,d001,110022,Margareta,Markovitch
1,d001,110039,Vishwani,Minakawa
2,d002,110085,Ebru,Alpin


In [23]:
# 5. List first name, last name, and sex for employees whose first name is "Hercules" and last names begin with "B."
query_5 = """
SELECT d.dept_no, dp.emp_no, e.first_name, e.last_name 
FROM departments d
    JOIN dept_manager dp ON d.dept_no = dp.dept_no 
    JOIN employees e ON e.emp_no = dp.emp_no
GROUP BY d.dept_no, dp.emp_no, e.emp_no;
"""

data_5 = pd.read_sql(query_5, conn)
data_5.head(3)

,dept_no,emp_no,first_name,last_name
0,d001,110022,Margareta,Markovitch
1,d001,110039,Vishwani,Minakawa
2,d002,110085,Ebru,Alpin
3,d002,110114,Isamu,Legleitner
4,d003,110183,Shirish,Ossenbruggen
5,d003,110228,Karsten,Sigstam
6,d004,110303,Krassimir,Wegerle
7,d004,110344,Rosine,Cools
8,d004,110386,Shem,Kieras
9,d004,110420,Oscar,Ghazalie
